# Base Image Generation Module


In [1]:
import torch
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler, AutoPipelineForText2Image, ControlNetModel, StableDiffusionXLControlNetPipeline
import torch

controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16
)

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "lykon/dreamshaper-xl-v2-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    controlnet=controlnet
).to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, algorithm_type="sde-dpmsolver++", use_karras_sigmas=True)

pipe.enable_model_cpu_offload()

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-03-19 03:49:10.127725: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 03:49:10.134584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-19 03:49:10.143015: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one h

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [41]:
import os
import random
import json
import torch
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image
from diffusers.utils import make_image_grid, load_image

def generate_base_images(
    in_folder: str,
    out_folder: str,
    prompt_pairs: list[tuple[str, str, str, int]],  # [(prompt, negative_prompt), ...]
    model_name: str = "model_name",
    show: bool = False
):
    """
    For each image in 'in_folder', and for each (prompt, negative_prompt) pair in prompt_pairs,
    generate 'how_many' images. For each generated image:

    - Save the result as: "{base_name}__{pair_index}_{gen_index}.png"
    - Save a matching JSON file: "{base_name}__{pair_index}_{gen_index}.json"
      with keys: prompt, negative_prompt, seed, model.

    :param in_folder: Folder with the input images.
    :param out_folder: Where we save generated images & JSON metadata.
    :param prompt_pairs: List of (prompt, negative_prompt) pairs.
    :param how_many: Number of images to generate per prompt pair.
    :param model_name: Model name or identifier to include in JSON metadata.
    :param show: If True, display the results in a grid (for debug).
    """
    os.makedirs(out_folder, exist_ok=True)

    valid_exts = (".jpg", ".jpeg", ".png", ".bmp")
    image_files = [f for f in os.listdir(in_folder) if f.lower().endswith(valid_exts)]

    already_generated_files = set([f for f in os.listdir(out_folder) if f.lower().endswith(valid_exts)])

    for img_name in tqdm(image_files, desc="Processing images", unit="img"):
        base_name, ext = os.path.splitext(img_name)

        image_path = os.path.join(in_folder, img_name)

        json_path = os.path.join(in_folder, f"{base_name}.json")
        if not os.path.exists(json_path):
            print(f"Skipping {img_name}: no matching JSON {base_name}.json found.")
            continue
        
        image_bgr = cv2.imread(image_path)
        if image_bgr is None:
            print(f"Skipping invalid file: {img_name}")
            continue

        image_bgr = cv2.resize(image_bgr, (1024, 1024))
        image_pil = Image.fromarray(image_bgr)

        with open(json_path, "r") as f:
            label_data = json.load(f)

        all_generated = [image_pil]

        for (variant, prompt, negative_prompt, how_many) in prompt_pairs:
            for gen_index in range(how_many):
                out_img_name = f"{base_name}__{variant}_{gen_index}.png"
                out_json_name = f"{base_name}__{variant}_{gen_index}.json"

                if out_img_name in already_generated_files:
                    print(f"Already generated {out_img_name}, skipping...")
                    continue

                seed = random.randint(0, 2**31 - 1)
                generator = torch.Generator(device=pipe.device).manual_seed(seed)

                result = pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    image=image_pil,
                    generator=generator,
                    num_inference_steps=20,
                    guidance=2,
                )
                gen_image = result.images[0]


                out_img_path = os.path.join(out_folder, out_img_name)
                gen_image.save(out_img_path)

                new_label_data = {
                    **label_data,
                    "variant": variant,
                    "prompt": prompt,
                    "negative_prompt": negative_prompt,
                    "seed": seed,
                    "model": model_name
                }

                out_json_path = os.path.join(out_folder, out_json_name)
                with open(out_json_path, "w") as f:
                    json.dump(new_label_data, f, indent=4)

                all_generated.append(gen_image)

        if show:
            # 1 row, number of columns = 1 + (#prompts * how_many)
            grid = make_image_grid(all_generated, rows=1, cols=len(all_generated))
            grid.show()


In [39]:
base_prompt = "photo of airport runway, aerial view, 4k, cinematic film still, realistic, beautiful landscape around, high-contrast runway lines"
base_neg = "airplane, ugly, low-quality, ugly background, ugly airstrip, deformed, dark, noisy, blurry, low contrast, missing lines, unrealistic, drawing, objects on runway"

modifiers = {
    "rain": (
        "+raining +storm +wet",
        "-dark -noisy -blurry",
    ),
    "fog": (
        "+(harsh fog) +mist +haze",
        "-dark -noisy -blurry",
    ),
    "snow": (
        "+snowing",
        "",
    ),
    "dusk": (
        "+(at dusk)",
        "",
    ),
    "dawn": (
        "+(at dawn)",
        "",
    ),
    "night": (
        "+(at night)",
        "-dark",
    )
}

import re

def _parse_modifiers(mod_string: str):
    """
    Example: '+(harsh fog) +mist +haze' -> ['+(harsh fog)', '+mist', '+haze']
    """
    return re.findall(r'([+\-][^+\-]+)', mod_string)


def _remove_parentheses(text: str) -> str:
    """
    E.g. '(harsh fog)' -> 'harsh fog'
    """
    return text.replace("(", "").replace(")", "")


def apply_modifiers(name, chosen_modifiers, count=1):
    """
    Modifies a base (prompt, negative_prompt) by applying the selected modifiers.
    Removes any parentheses in the tokens, e.g. '+(at dusk)' -> 'at dusk'.

    :param name: A string you want returned (unused internally).
    :param chosen_modifiers: List of modifier keys to apply in sequence.
    :param count: An integer returned at the end (for your own usage).
    :return: (name, final_prompt, final_neg, count)
    """
    prompt_tokens = [p.strip() for p in base_prompt.split(",") if p.strip()]
    neg_tokens = [n.strip() for n in base_neg.split(",") if n.strip()]

    for mod_name in chosen_modifiers:
        if mod_name not in modifiers:
            continue
        
        prompt_mod, neg_mod = modifiers[mod_name]
        
        for token in _parse_modifiers(prompt_mod):
            token_stripped = token[1:].strip()  # remove leading '+' or '-'
            token_stripped = _remove_parentheses(token_stripped)
            if token.startswith("+"):
                prompt_tokens.append(token_stripped)
            elif token.startswith("-"):
                prompt_tokens = [p for p in prompt_tokens if p != token_stripped]

        for token in _parse_modifiers(neg_mod):
            token_stripped = token[1:].strip()
            token_stripped = _remove_parentheses(token_stripped)
            if token.startswith("+"):
                neg_tokens.append(token_stripped)
            elif token.startswith("-"):
                neg_tokens = [n for n in neg_tokens if n != token_stripped]

    final_prompt = ", ".join(prompt_tokens)
    final_neg = ", ".join(neg_tokens)

    return name, final_prompt, final_neg, count


apply_modifiers("night+fog", ["night", "fog"], 1)

('night+fog',
 'photo of airport runway, aerial view, 4k, cinematic film still, realistic, beautiful landscape around, high-contrast runway lines, at night, harsh fog, mist, haze',
 'airplane, ugly, low-quality, ugly background, ugly airstrip, deformed, low contrast, missing lines, unrealistic, drawing, objects on runway',
 1)

In [ ]:
generate_base_images(
    "p_FilteredCannyEdges",
    "p_BaseImages",
    prompt_pairs=[
        apply_modifiers("day", [], 5),
        apply_modifiers("night", ["night"], 5),
        apply_modifiers("dusk", ["dusk"], 1),
        apply_modifiers("dawn", ["dawn"], 1),
        apply_modifiers("fog", ["fog"], 1),
        apply_modifiers("fog+night", ["night", "fog"], 1),
        apply_modifiers("rain", ["rain"], 1),
        apply_modifiers("rain+night", ["night", "rain"], 1),
        apply_modifiers("snow", ["snow"], 1),
        apply_modifiers("snow+night", ["night", "snow"], 1),
    ],
    model_name="sdxl-dreamshaperxl",
    # show=True
)

Processing images:   0%|          | 0/361 [00:00<?, ?img/s]

Already generated 82ui7mLMp5o_037__day_0.png, skipping...
Already generated 82ui7mLMp5o_037__day_1.png, skipping...
Already generated 82ui7mLMp5o_037__day_2.png, skipping...
Already generated 82ui7mLMp5o_037__day_3.png, skipping...
Already generated 82ui7mLMp5o_037__day_4.png, skipping...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Processing images:  20%|██        | 73/361 [2:27:13<9:40:57, 121.03s/img]

Already generated AizLOuimGXU_032__day_0.png, skipping...
Already generated AizLOuimGXU_032__day_1.png, skipping...
Already generated AizLOuimGXU_032__day_2.png, skipping...
Already generated AizLOuimGXU_032__day_3.png, skipping...
Already generated AizLOuimGXU_032__day_4.png, skipping...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]